<a href="https://colab.research.google.com/github/ijul17/homework/blob/main/Yulia%20Urazova/For_StreamLit_Legprom_last_data_Tfidf_model_with_2_fabrics_4_cols_reduced_with_comments_%D0%B4%D0%BE%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from gensim import corpora,models,similarities
from gensim.utils import tokenize
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#import gdown
#F_URL = "https://docs.google.com/spreadsheets/d/1uoj0R7e6pTlX9EK2CvdVOt1FePBIVYyj/edit?usp=sharing&ouid=118340160593820077479&rtpof=true&sd=true" #файл с фабриками, заказами

#file_id = F_URL.split('/')[-2]
#id="https://drive.google.com/uc?id=" + file_id


In [2]:
#начальный словарь, полученный на вход
dict_order = {"Вид изделия": "Одежда",
                      "Тип одежды": "Трикотажная одежда, Домашняя одежда",
                      #"Название": "Костюм трикотажный",
                      "Назначение": "Повседневная одежда",
                      "По полу и возрасту": "Женская одежда",
                      "Сезон": "Всесезон",
                      "Ценовой сегмент": "Средний, Средний Плюс",
                      "Заказчик предоставит": "Лекала, Фото изделия",
                      "Конструирование": "Требуется разработка лекал",
                      "Виды нанесения": "Бирка, Штамп",
                      "Требования к оборудованию": "Пуговичная машина с приспособлением для пуговиц на ножке",
                      "Комментарий к заказу": "Просим изготовить домашние костюмы для женщин от 50 лет и старше. Размерный ряд от 48 до 60",
                      "Регионы производства": "ЦФО", "Образец":"Да", "Пробная партия":"50 единиц",
                      "Плановый бюджет":"350000",
                      "Наличие в реестре Минпромторга": "Да",
                      "Требование к штату":"Не менее 8 человек в бригаде. Технолог швейного производства в штате",
                      "Упаковка":"Лейблы и бирки",
                      "Условия оплаты":"Аванс 50%",
                      "Дополнительные услуги": "ВТО, упаковка",
                      "Сертификация":"Обычная одежда, не требует обязательной сертификации",
                      "Обеспечение сырьем":"Закупает фабрика"
                    }

In [3]:
from google.colab import files
tables=files.upload()

Saving ТЗ для Датасетов доп фабрики.xlsx to ТЗ для Датасетов доп фабрики.xlsx


In [4]:
xl = pd.ExcelFile('ТЗ для Датасетов доп фабрики.xlsx')
df_fabrics = xl.parse('Фабрики')


#	'По сезону',

In [11]:
          #Подготовка данных по фабрикам
df_fabrics_min=df_fabrics[['ID пользователя','Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'Ценовые сегменты',	'Регион производства',
                                                 'Конструирование','Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка']].copy()


In [12]:
df_fabrics_min = df_fabrics_min.astype(str)

In [13]:
df_fabrics_min['fabrics_all_info'] = df_fabrics_min[['Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'Ценовые сегменты',	'Регион производства',
                                                 'Конструирование','Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка']].agg(' '. join , axis= 1 )

In [14]:

df_fabrics_min.drop(['Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'Ценовые сегменты',	'Регион производства',
                                                 'Конструирование','Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка'], axis=1, inplace= True)

In [15]:
fabrics_info=df_fabrics_min['fabrics_all_info'].str.replace('nan', '')  #удалим nan из описания
fabrics_info=df_fabrics_min['fabrics_all_info'].str.replace(',', '')
fabrics_info=df_fabrics_min['fabrics_all_info'].str.lower()

In [16]:


list_values_for_new_dict = set(('Вид изделия','Название','Тип одежды','Назначение','Комментарий к заказу'))
order_for_model = {k:v for k,v in dict_order.items() if k in list_values_for_new_dict} #заказ в модель
fix_features = [k for k,v in dict_order.items() if k not in list_values_for_new_dict]  #неиспользуемые в модели поля для заказа

order_min = pd.DataFrame(list(order_for_model.values ()))
order_min=order_min.transpose()  #транспонируем
order_min.columns = list(order_for_model.keys())  #назовем столбцы по названию полей заказа

            #объединим данные в строку
order_min['orders_all_info'] = order_min. agg(' '. join , axis= 1 )

            #преобразуем - уберем пустые значения, приведем к нижнему регистру
order_min['orders_all_info']=order_min['orders_all_info'].str.replace('nan', '')  #удалим nan из описания
order_min['orders_all_info']=order_min['orders_all_info'].str.replace(',', '')
order_min['orders_all_info']=order_min['orders_all_info'].str.lower()

            #выделим описания, по которым будем сравнивать заказы
text_to_compare = order_min['orders_all_info']
# импортируем класс для лемматизации
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# создаём объект этого класса
lemmatizer = WordNetLemmatizer()

# и пустой список для слов после лемматизации
lemmatized = []

# проходимся по всем токенам
for token in text_to_compare:

    # применяем лемматизацию
    token_order = lemmatizer.lemmatize(token)

    # добавляем слово после лемматизации в список
    lemmatized.append(token)

#выделим основы слов в описании заказа
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
orders_stemmed=[]  #сюда сформируем новый перечень описаний фабрик - только основы слов
for str in lemmatized:
 stemmed_words_orders = [stemmer.stem(word) for word in str.split()]
 row_stemmed_order = ' '.join(stemmed_words_orders)
 orders_stemmed.append(row_stemmed_order)





          #'''Функция предобработки данных из файла фабрик.'''

#xl = pd.ExcelFile('ТЗ для Датасетов доп фабрики.xlsx')
#df_fabrics = xl.parse('Фабрики')


          #Подготовка данных по фабрикам

          #Подготовка данных по фабрикам
#df_fabrics_min=df_fabrics[['Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'Ценовые сегменты',	'Регион производства',
                                                # 'Конструирование','Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка']].copy()


#df_fabrics_min = df_fabrics_min.astype(str)

#df_fabrics_min['fabrics_all_info'] = df_fabrics_min.agg(' '. join , axis= 1 )

#df_fabrics_min(['Регистрация',	'Логотип',	'Название',	'Email',	'Телефон',	'Сайт',	'Имя контактного лица', 'Тип работ', 'Технология','Пошив образцов',
                     #'Минимальная партия, шт.','Минимальная сумма заказа, руб','Условия оплаты',	'Заказчик должен предоставить',
                     #'Дополнительные услуги',	'Сертификация',	'Загруженность','Наличии в реестре Минпромторга'], axis= 1 , inplace= True)


#df_fabrics_min.drop(['Вид изделий',	'Тип одежды',	'Сфера применения',	'По полу/возрасту',	'Ценовые сегменты',	'Регион производства',
                                                 #'Конструирование','Сырье',	'Нанесение',	'Оборудование',	'Штат',	'Упаковка'], axis=1, inplace= True)


           #преобразуем - уберем пустые значения, приведем к нижнему регистру


#fabrics_info=df_fabrics_min['fabrics_all_info'].str.replace('nan', '')  #удалим nan из описания
#fabrics_info=df_fabrics_min['fabrics_all_info'].str.replace(',', '')
#fabrics_info=df_fabrics_min['fabrics_all_info'].str.lower()


       #Лемматизация импортируем класс для лемматизации

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
           # создаём объект этого класса
lemmatizer = WordNetLemmatizer()

          # и пустой список для слов после лемматизации
lemmatized = []

          # проходимся по всем токенам
for token in df_fabrics_min['fabrics_all_info']:

          # применяем лемматизацию
  token = lemmatizer.lemmatize(token)

          # добавляем слово после лемматизации в список
  lemmatized.append(token)




         ##Стеммизация - выделим основы слов в описании фабрик
import nltk
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
fabrics_stemmed=[]  #сюда сформируем новый перечень описаний фабрик - только основы слов
for str in lemmatized:
  stemmed_words = [stemmer.stem(word) for word in str.split()]
  row_stemmed = ' '.join(stemmed_words)
  fabrics_stemmed.append(row_stemmed)
         #добавим сокращенные описания в датафрейм
df_fabrics_min["reduced_desript"]=fabrics_stemmed


       #токенизируем все сокращенные тексты в нашем датасете
def tokenize_in_df(strin):
          try:
             return list(tokenize(strin, deacc=True,))
          except:
             return ""
df_fabrics_min["tokens"] = df_fabrics_min["reduced_desript"].apply(tokenize_in_df)

      #Создадим словарь слов, которые есть во всем нашем наборе текстов:
dictionary = corpora.Dictionary(df_fabrics_min["tokens"])
feature_cnt = len(dictionary.token2id)
      #dictionary.token2id

      #Создаем корпус, превратив наши токенизированные тексты в векторы
corpus = [dictionary.doc2bow(text) for text in df_fabrics_min["tokens"]]

      # модель tf-idf
tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(tfidf[corpus],num_features = feature_cnt)
      #векторы наших опорных текстов и получив значения их похожестей из матрицы.
for text in orders_stemmed:
  kw_vector = dictionary.doc2bow(tokenize(text))
  df_fabrics_min[text] = index[tfidf[kw_vector]]


      #можем избавиться от заведомо непохожих текстов, посчитав сумму весов и оставив тексты с самыми высокими суммами
df_fabrics_min["sum"] = 0
for text in orders_stemmed:
   df_fabrics_min["sum"] = df_fabrics_min["sum"]+df_fabrics_min[text]

      #Избавляться от лишних текстов можно обрезав по порогу суммы, или отсортировав по сумме и обрезав датасет по количеству текстов.
df_fabrics_min["sum"].value_counts(bins=5)

df_fabrics_results=df_fabrics_min[df_fabrics_min["sum"]>0.02]
df_fabrics_results = df_fabrics_results.sort_values(by='sum', ascending=False)
id_fabrics=df_fabrics_results['ID пользователя'].to_list()

dict_for_code={'companies':id_fabrics, 'fix_features': fix_features}



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
